In [ ]:
%matplotlib notebook

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re  #for regular expression
import nltk #for text manipulation
import string
import warnings
import numpy as np
import pandas as pd

pd.set_option("display.max_colwidth",200)

%matplotlib inline

In [ ]:
train = pd.read_csv("../input/twitter-sentiment-analysis-hatred-speech/train.csv")
test = pd.read_csv("../input/twitter-sentiment-analysis-hatred-speech/test.csv")
train[train['label']== 0].head(10)

In [ ]:
train[train['label']== 1].head(10)

In [ ]:
train.shape,test.shape

In [ ]:
train["label"].value_counts()

In [ ]:
combi = train.append(test, ignore_index=True)
combi.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer 
import gensim
bow_vectorizer = CountVectorizer(max_df=0.90 ,min_df=2,max_features=1000,stop_words='english')
bow = bow_vectorizer.fit_transform(combi['tweet'])
bow.shape



In [ ]:
tfdif_vectorizer = TfidfVectorizer(max_df=0.90 ,min_df=2,max_features=1000,stop_words='english')
tfidf = tfdif_vectorizer.fit_transform(combi['tweet'])
tfidf.shape

In [ ]:
tokenized_tweet = combi['tweet'].apply(lambda x: x.split()) #tokenizing
model_w2v = gensim.models.Word2Vec(tokenized_tweet,vector_size = 200 ,window=5,min_count=2,sg=1,hs=0,negative=10,workers= 2,seed = 34)
model_w2v.train(tokenized_tweet, total_examples=len(combi['tweet']),epochs=20)


In [ ]:
model_w2v.wv.most_similar(positive='dinner')

In [ ]:
model_w2v.wv.most_similar(negative='trump')

In [ ]:

vector = model_w2v.wv['food']
print(vector)
len(model_w2v.wv['food'])

In [ ]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v.wv[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
wordvec_arrays = np.zeros((len(tokenized_tweet),200))
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i],200)
    wordvec_df = pd.DataFrame(wordvec_arrays)
    wordvec_df.shape

In [ ]:
wordvec_df.shape

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score

In [ ]:
train_bow = bow[:31962,:]
test_bow = bow[31962:,:]
xtrain_bow,xvalid_bow,ytrain,yvalid = train_test_split(train_bow,train['label'],random_state = 411,test_size=0.3)

lreg = LogisticRegression()
lreg.fit(xtrain_bow, ytrain)
prediction = lreg.predict_proba(xvalid_bow)
predict_int = prediction[:,1] >= 0.3 
predict_int = predict_int.astype(np.int)
f1_score(yvalid,predict_int)



In [ ]:
test_pred = lreg.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)
test['label'] = test_pred_int 
submission = test[['id','label']]
submission.to_csv('sub_lreg_bow.csv',index=False)


In [ ]:
train_w2v = wordvec_df.iloc[:31962,:]
test_w2v = wordvec_df.iloc[31962:,:]
xtrain_w2v = train_w2v.iloc[ytrain.index,:]
xvalid_w2v = train_w2v.iloc[yvalid.index,:]

lreg.fit(xtrain_w2v, ytrain)
prediction = lreg.predict_proba(xvalid_w2v)
predict_int = prediction[:,1] >= 0.3 
predict_int = predict_int.astype(np.int)
f1_score(yvalid,predict_int)


In [ ]:
train_tfidf = tfidf[:31962,:]
test_tfidf = tfidf[31962:,:]
xtrain_tfidf = train_tfidf[ytrain.index]
xvalid_tfidf = train_tfidf[yvalid.index]

lreg.fit(xtrain_tfidf, ytrain)
prediction = lreg.predict_proba(xvalid_tfidf)
predict_int = prediction[:,1] >= 0.3 
predict_int = predict_int.astype(np.int)
f1_score(yvalid,predict_int)

In [ ]:
from sklearn import 